<a href="https://colab.research.google.com/github/kiran-lenka/Bitcoin-Price-Predictor/blob/main/BitCoin_Major_Project_Sentiment_Analysis_and_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing Requirements

In [ ]:

# importing the necessary libraries
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import ssl  
import os
from datetime import date
from io import StringIO
import json

import nltk 
nltk.download('wordnet')
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.svm import NuSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import AdaBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn import metrics

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


# Bitcoin Market Sentiment Analysis and Prediction

## Calculating public sentiment on Bitcoin

### Reddit Sentiment Analysis

In [ ]:
# grouping post by sentiment
df_reddit_nlp = pd.read_csv('reddit-senti.csv')
df_reddit_groupby = df_reddit_nlp.groupby('Sentiment').count()
sentiment_lst = ['Negative', 'Neutral', 'Positive']
lst1 = df_reddit_groupby['Post Titles']
dict1 = {'Sentiment': sentiment_lst, 'Number': lst1}
sent = pd.DataFrame(dict1)
fig = px.bar(sent, x='Sentiment', y='Number', title='Reddit Sentiment Analysis')
fig.show()

### Twitter Sentiment Analysis

In [ ]:
types = {'Tweets':'str','Sentiment':'str'}
df_twitter_nlp = pd.read_csv('twitter-senti.csv',index_col=0,dtype=types)

In [ ]:
df_twitter_nlp

,neg,neu,pos,compound,Tweets,Sentiment,Created at,Location
0,0.000,1.0,0.000,0.0000,@sanazmsvi @nftparticle #Flokifam\nThe land of...,0,2022-01-06 11:19:59,NaN
1,0.292,0.708,0.000,-0.7650,RT @NFTmagzine: #Sheepzy died for all of the #...,-1,2022-01-06 11:19:59,NaN
2,0.000,0.854,0.146,0.4738,RT @Zbaaboulinet: Scholarship Giveaway ! 50/50...,1,2022-01-06 11:19:59,elbi ✨
3,0.222,0.672,0.106,-0.5319,RT @Fantasywar2022: FANTASY WAR PRESALE WHITEL...,-1,2022-01-06 11:19:59,Việt Nam
4,0.000,1.0,0.000,0.0000,GM #nft people ☀️📈 #gm #NFTartist #Crypto,0,2022-01-06 11:19:59,NaN
...,...,...,...,...,...,...,...,...
7995,0.000,0.891,0.109,0.4215,RT @JoshuaAbel68: @AltCryptoGems You will be a...,1,2022-01-03 19:25:13,Gwagwalada - Abuja
7996,0.000,0.891,0.109,0.4215,RT @JoshuaAbel68: @AltCryptoGems You will be a...,1,2022-01-03 19:24:47,NaN
7997,0.000,0.773,0.227,0.5719,"@ForbesCrypto hidden gem, thank me later @Devo...",1,2022-01-03 19:24:43,unknown
7998,0.000,0.891,0.109,0.4215,RT @JoshuaAbel68: @AltCryptoGems You will be a...,1,2022-01-03 19:24:30,NaN


In [ ]:
# grouping post by sentiment
df_twitter_groupby = df_twitter_nlp.groupby('Sentiment').count()
lst1 = df_twitter_groupby['Tweets']
sentiment_lst = ['Negative', 'Neutral', 'Positive']
dict1 = {'Sentiment': sentiment_lst, 'Number': lst1}
sent = pd.DataFrame(dict1)
fig = px.bar(sent, x='Sentiment', y='Number', title='Twitter Sentiment Analysis')
fig.show()

## Preprocessing and Building Models

In [ ]:
stemmer = SnowballStemmer("english")

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# Tokenize and lemmatize
def preprocess(text):
    
    result = []
    
    for token in gensim.utils.simple_preprocess(text) :
        
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            
            # TODO: Apply lemmatize_stemming() on the token, then add to the results list
            result.append(lemmatize_stemming(token))
    
    return result

def classifier_models():
  LR = LogisticRegression(random_state=0)
  model_perf(LR, 'Logistic Regression')

  RFC = RandomForestClassifier(criterion = 'gini')
  model_perf(RFC, 'Random Forest')

  SGDC = SGDClassifier(max_iter=1000, tol=1e-3)
  model_perf(SGDC, 'Stochastic Gradient Descent')

  svm = SVC()
  model_perf(svm, 'Support Vector Machine')
  '''
  LSVC = LinearSVC(random_state=0, tol=1e-5)
  model_perf(LSVC, 'Linear Support Vector Classification')

  GNB = GaussianNB()
  model_perf(GNB, 'Naive Bayes')  

  BNB = BernoulliNB()
  model_perf(BNB, 'Bernoulli Naive Bayes')

  MNB = MultinomialNB()
  model_perf(MNB, 'Multinomial Naive Bayes')

  AdaB = AdaBoostClassifier(n_estimators=100, random_state=0)
  model_perf(AdaB, 'AdaBoost')
  '''

  LGBM = LGBMClassifier(random_state=5)
  model_perf(LGBM, 'Light Gradient Boosting Machine')

  GBC = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
  model_perf(GBC, 'Gradient Boost Classifier')

## Performance Comparison between different classifiers

### Reddit

In [ ]:
# passing reddit posts through various classifiers

name_arr = []
acc_arr = []
  
def model_perf(model, name):
  model.fit(X_train, y_train)
  preds = model.predict(X_test)
  acc = round(accuracy_score(y_test, preds), 5)*100
  name_arr.append(name)
  acc_arr.append(acc)
  print('======================================')
  print('Accuracy ' + f'{name}' + ':', acc,'\n')
  print(classification_report(y_test, preds))
  print('======================================')

processed_docs = df_reddit_nlp['Post Titles'].map(preprocess)

# this is the step where each word is given a number from which it can be recognized
tokenizer = Tokenizer(oov_token='<00V>')
tokenizer.fit_on_texts(processed_docs)
word_index = tokenizer.word_index

# here the tokenized numbers are put into sequences as compared to the post titles
sequences = tokenizer.texts_to_sequences(processed_docs)

# here we make all the sentences of equal length
padded = pad_sequences(sequences, padding='post')

X = padded
y = df_reddit_nlp['Sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

classifier_models()

Accuracy Logistic Regression: 56.908 

              precision    recall  f1-score   support

          -1       0.48      0.03      0.05       438
           0       0.59      0.91      0.71      1389
           1       0.49      0.28      0.35       793

    accuracy                           0.57      2620
   macro avg       0.52      0.40      0.37      2620
weighted avg       0.54      0.57      0.49      2620

Accuracy Random Forest: 82.786 

              precision    recall  f1-score   support

          -1       0.92      0.66      0.77       438
           0       0.83      0.90      0.86      1389
           1       0.79      0.79      0.79       793

    accuracy                           0.83      2620
   macro avg       0.85      0.78      0.81      2620
weighted avg       0.83      0.83      0.83      2620

Accuracy Stochastic Gradient Descent: 52.71 

              precision    recall  f1-score   support

          -1       0.23      0.16      0.19       438
           

In [ ]:
dict2 = {'Name of the models': name_arr, 'Accuracy': acc_arr}
perf = pd.DataFrame(dict2)
fig = px.bar(perf, x='Name of the models', y='Accuracy', title='Accuracy scores of various classifiers models')
fig.show()

### Twitter

In [ ]:
# passing twitter posts through various classifiers

name_arr = []
acc_arr = []
  
def model_perf(model, name):
  model.fit(X_train, y_train)
  preds = model.predict(X_test)
  acc = round(accuracy_score(y_test, preds), 5)*100
  name_arr.append(name)
  acc_arr.append(acc)
  print('======================================')
  print('Accuracy ' + f'{name}' + ':', acc,'\n')
  print(classification_report(y_test, preds))
  print('======================================')

df_twitter_nlp.replace([np.inf, -np.inf], np.nan, inplace=True)
df_twitter_nlp.fillna('', inplace=True)
processed_docs = df_twitter_nlp['Tweets'].map(preprocess)

# this is the step where each word is given a number from which it can be recognized
tokenizer = Tokenizer(oov_token='<00V>')
tokenizer.fit_on_texts(processed_docs)
word_index = tokenizer.word_index

# here the tokenized numbers are put into sequences as compared to the post titles
sequences = tokenizer.texts_to_sequences(processed_docs)

# here we make all the sentences of equal length
padded = pad_sequences(sequences, padding='post')

X = padded
y = df_twitter_nlp['Sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

classifier_models()

Accuracy Logistic Regression: 46.858 

              precision    recall  f1-score   support

                   0.00      0.00      0.00         3
          -1       0.50      0.01      0.02       257
           0       0.41      0.52      0.46       918
           1       0.52      0.53      0.52      1225

    accuracy                           0.47      2403
   macro avg       0.36      0.26      0.25      2403
weighted avg       0.48      0.47      0.45      2403

Accuracy Random Forest: 83.937 

              precision    recall  f1-score   support

                   1.00      0.67      0.80         3
          -1       0.98      0.59      0.73       257
           0       0.81      0.84      0.83       918
           1       0.84      0.89      0.87      1225

    accuracy                           0.84      2403
   macro avg       0.91      0.75      0.81      2403
weighted avg       0.85      0.84      0.84      2403

Accuracy Stochastic Gradient Descent: 36.912 

           

In [ ]:
dict3 = {'Name of the models': name_arr, 'Accuracy': acc_arr}
perf1 = pd.DataFrame(dict3)
fig = px.bar(dict3, x='Name of the models', y='Accuracy', title='Accuracy scores of various classifiers models')
fig.show()